In [ ]:
# Transformer 모델 구축 - Transformer 질의응답(QA) 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용

In [ ]:
# QA Pre-trained 모델 테스트
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

base_model = 'monologg/koelectra-base-v3-finetuned-korquad' # 한국어 KorQuAD 데이터셋으로 파인 튜닝된 KoELECTRA QA 모델
tokenizer = AutoTokenizer.from_pretrained(base_model) # 토크나이저 로드
model = AutoModelForQuestionAnswering.from_pretrained(base_model) # QA 태스크용 헤드가 포함된 모델 로드

question = '서울은 어디에 있나요?'
context = '서울은 대한민국의 수도이며, 한반도의 중서부에 위치해 있습니다.'

# QA 모델 구조: BERT 기반 QS 모델은 입력을 [CLS]질문[SEP]문맥[SEP] 형태로 받는다
# 질문과 문맥을 하나의 입력으로 합쳐서 토큰화 결과를 모델에 전달해야 답변을 얻을 수 있는 구조
# [CLS] 서울 은 어디 에 있 나요 ? [SEP] 서울 은 대한민국 의 수도 이며 , 한반도 의 중서부 에 위치 해 있습니다 . [SEP]
inputs = tokenizer(question, context, return_tensors='pt') # (batch_size,seq_len) (1,seq_len)

# 모델에 입력을 전달하여 start_logits, end_logits 출력
# - start_logits: 답변 시작 위치에 대한 확률 분포
# - end_logits: 답변 끝 위치에 대한 확률 분포
outputs = model(**inputs)

# torch.argmax() 가장 확률이 높은 인덱스를 선택
# - answer_start: 답변 시작 토큰 위치, answer_end: 답변 끝 토큰 위치
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1 # 마지막 인덱스 포함

# 토큰을 문자열로 변환
# - tokenizer.convert_ids_to_tokens(): 토큰 ID->토큰 문자열
# - tokenizer.convert_tokens_to_string(): 토큰 문자열->사람이 읽을 수 있는 문장
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end])
)

# Debug 확인
print('답변 시작 인덱스:', answer_start.item())
print('답변 끝 인덱스:', answer_end.item())

tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]) # 전체 토큰 확인
print('전체 토큰 시퀀스:', tokens)

answer_tokens = tokens[answer_start:answer_end] # 답변 토큰 범위 확인
print('답변 토큰:', answer_tokens)

answer = tokenizer.convert_tokens_to_string(answer_tokens) # 최종 답변
print('최종 답변:', answer)

In [ ]:
# 데이터셋 로드 및 전처리
# - KorQuAD 1.0/2.0: 위키 문서 기반, 질문-답변 쌍 포함, AI Hub QA 데이터(한국어 QA 태스크용)
# - SQuAD 1.1/2.0: 가장 널리 쓰이는 QA 데이터셋
from datasets import load_dataset
from transformers import AutoTokenizer

# 한국어 KorQuAD 데이터셋: train(60,407), validation(5,774)
# - 데이터 구조: context(문서 본문), question(질문), answers(정답 스팬, 텍스트 + 시작 위치)
dataset = load_dataset('squad_kor_v1')

# validation 데이터셋을 반으로 나누어 test 데이터셋 생성
split_dataset = dataset['validation'].train_test_split(test_size=0.5, seed=42)
dataset['validation'] = split_dataset['train']
dataset['test'] = split_dataset['test']
print(dataset['train'].shape, dataset['validation'].shape, dataset['test'].shape)

In [ ]:
# 토크나이저
model_name = 'monologg/koelectra-base-v3-finetuned-korquad'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 전처리 함수
def preprocess(example):
    return tokenizer(
        example['question'],
        example['context'],
        truncation=True,
        padding='max_length',
        max_length=512
    )
tokenized_dataset = dataset.map(preprocess, batched=True)

In [ ]:
# collate_fn 정의 및 DataLoader 생성
from torch.utils.data import DataLoader

# collate_fn 정의: 텐서 변환
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(x['input_ids']) for x in batch])
    attention_mask = torch.stack([torch.tensor(x['attention_mask']) for x in batch])
    start_positions = torch.stack([torch.tensor(x['start_positions']) for x in batch])
    end_positions = torch.stack([torch.tensor(x['end_positions']) for x in batch])
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'start_positions': start_positions,
        'end_positions': end_positions
    }

# DataLoader 구성: train, validation, test
train_loader = DataLoader(
    tokenized_dataset['train'], 
    batch_size=16, 
    shuffle=True,
    collate_fn=collate_fn
)
valid_loader = DataLoader(
    tokenized_dataset['validation'], 
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    tokenized_dataset['test'], 
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn
)